In [1]:
from datasets import load_dataset

d:\app\anaconda\envs\webui\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import random

In [12]:
a = [1, 2, 3, 4, 100]
# shuffle but not in place
b = random.sample(a, len(a))

In [13]:
b

[4, 1, 3, 100, 2]

In [ ]:
import torch
from torchmetrics.functional import stat_scores
from torchmetrics import Accuracy

# 假设 y_true 和 y_pred 是 PyTorch tensor 格式的标签
y_true = torch.tensor([[1, 0, 1], [0, 1, 1], [1, 1, 0]])
y_pred = torch.tensor([[1, 1, 0], [0, 1, 1], [1, 1, 0]])

# 使用 TorchMetrics 计算多标签F1分数
acc = Accuracy(num_classes=3, task="binary", num_labels=3, average="none")
f1 = acc(y_pred, y_true)
print(f"F1 Score: {f1}")

In [2]:
from datasets import load_dataset

ds = load_dataset("E:/pretrained_models/datasets/figer/figer/1.0.0")

Generating train split: 1505765 examples [00:00, 2467139.82 examples/s]
Generating test split: 434 examples [00:00, 216525.27 examples/s]


In [4]:
ds.push_to_hub("DGME/figer", token="")

Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  1.02it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/DGME/figer/commit/30d30988600b1aafe01208d48936680d1cce0b6a', commit_message='Upload dataset', commit_description='', oid='30d30988600b1aafe01208d48936680d1cce0b6a', pr_url=None, pr_revision=None, pr_num=None)

In [14]:
print(ds["train"][3]["mentions"])

{'start': [8, 13, 19], 'end': [11, 17, 23], 'labels': [['/train'], ['/organization', '/organization/company'], ['/rail/railway', '/organization', '/organization/company', '/rail', '/location']]}


In [15]:
print(ds["train"][:2]["tokens"])

[['It', 'includes', 'a', 'cover', 'of', '"', 'Low', 'Rider', '"', 'by', 'War', 'and', 'a', 'reworking', 'of', 'Muddy', 'Waters', "'", '"', 'I', 'Just', 'Wanna', 'Make', 'Love', 'To', 'You', '"', 'as', '"', 'I', 'Just', 'Wanna', 'Get', 'High', 'with', 'You', '"', '.'], ['He', 'was', 'hospitalized', 'for', 'four', 'days', 'at', 'the', 'Mayo', 'Clinic', 'in', '1964', ',', 'and', 'for', 'three', 'weeks', 'in', '1966', '.']]
